# Chapter 10: Collaboration - Part 2

## Items 88-90: Advanced Collaboration Techniques

This notebook covers three critical aspects of Python collaboration:
- Managing circular dependencies between modules
- Using warnings to guide API migrations
- Leveraging static type analysis to prevent bugs

---

## Item 88: Know How to Break Circular Dependencies

### The Problem: Circular Import Dependencies

Circular dependencies occur when two modules must call into each other at import time, causing import failures and runtime errors.

### Understanding Python's Import Mechanism

When Python imports a module, it performs these steps in depth-first order:

1. Searches for module in locations from `sys.path`
2. Loads code from module and ensures it compiles
3. Creates corresponding empty module object
4. Inserts module into `sys.modules`
5. Runs code in module object to define its contents

**The issue**: Module attributes aren't defined until step 5 completes, but the module can be imported after step 4.

### Example: Circular Dependency Problem

Let's demonstrate the circular dependency issue:

In [ ]:
# Simulating the circular dependency scenario
# In practice, this would be split across multiple files

# What dialog.py would look like:
circular_demo = '''
# dialog.py
import app

class Dialog:
    def __init__(self, save_dir):
        self.save_dir = save_dir

save_dialog = Dialog(app.prefs.get('save_dir'))  # Fails here!

def show():
    print(f"Showing dialog with: {save_dialog.save_dir}")
'''

# What app.py would look like:
app_demo = '''
# app.py
import dialog

class Prefs:
    def get(self, name):
        return "/tmp/documents"

prefs = Prefs()
dialog.show()
'''

print("Circular dependency structure:")
print("app.py imports dialog.py")
print("dialog.py imports app.py")
print("\nResult: AttributeError - 'app' module has no attribute 'prefs'")
print("(because app is still initializing when dialog tries to access it)")

### Solution 1: Reordering Imports (Not Recommended)

Move imports to the bottom of the file after other definitions.

In [ ]:
# app.py - with reordered import
reordered_example = '''
class Prefs:
    def get(self, name):
        return "/tmp/documents"

prefs = Prefs()

import dialog  # Moved to bottom
dialog.show()
'''

print("Reordering imports solution:")
print("✗ Violates PEP 8 style guide")
print("✗ Makes dependencies unclear")
print("✗ Brittle - small changes can break everything")
print("\n⚠️  NOT RECOMMENDED")

### Solution 2: Import, Configure, Run Pattern

Separate module initialization into distinct phases.

In [ ]:
# Demonstrating Import-Configure-Run pattern

# Simulated dialog module
class DialogModule:
    def __init__(self):
        self.Dialog = type('Dialog', (), {'__init__': lambda s, d=None: setattr(s, 'save_dir', d)})
        self.save_dialog = self.Dialog()
    
    def configure(self, prefs):
        """Configure after all modules are imported"""
        self.save_dialog.save_dir = prefs.get('save_dir')
    
    def show(self):
        print(f"Dialog showing: {self.save_dialog.save_dir}")

# Simulated app module
class AppModule:
    def __init__(self):
        self.Prefs = type('Prefs', (), {'get': lambda s, n: '/tmp/docs'})
        self.prefs = self.Prefs()
    
    def configure(self):
        """Configure after all modules are imported"""
        pass  # Could configure other settings

# Main program - three distinct phases
print("=== Phase 1: Import ===")
app = AppModule()
dialog = DialogModule()
print("All modules imported successfully")

print("\n=== Phase 2: Configure ===")
app.configure()
dialog.configure(app.prefs)
print("All modules configured")

print("\n=== Phase 3: Run ===")
dialog.show()

print("\n✓ Advantages:")
print("  - Clear separation of concerns")
print("  - Enables dependency injection")
print("\n✗ Disadvantages:")
print("  - Requires structural changes")
print("  - Separates definition from configuration")

### Solution 3: Dynamic Import (Recommended)

Import modules inside functions/methods when actually needed.

In [ ]:
# Demonstrating dynamic import pattern

class DynamicDialog:
    """Dialog using dynamic imports"""
    def __init__(self):
        self.save_dir = None
    
    def show(self, app_module):
        """Import happens at runtime, not initialization"""
        # In real code: import app
        # Here we pass it as parameter for demo
        self.save_dir = app_module.prefs.get('save_dir')
        print(f"Showing dialog: {self.save_dir}")

class DynamicApp:
    """App using dynamic dialog"""
    class Prefs:
        def get(self, name):
            return '/home/user/documents'
    
    def __init__(self):
        self.prefs = self.Prefs()
    
    def run(self, dialog_instance):
        dialog_instance.show(self)

# Usage
app = DynamicApp()
dialog = DynamicDialog()
app.run(dialog)

print("\n✓ Advantages of dynamic imports:")
print("  - Minimal code changes required")
print("  - No structural refactoring needed")
print("  - Works with existing code patterns")
print("\n⚠️  Considerations:")
print("  - Import has performance cost (negligible in most cases)")
print("  - Can delay SyntaxError discovery")
print("  - Use tests to catch delayed errors")

### Comparison of Solutions

In [ ]:
import pandas as pd

comparison_data = {
    'Solution': [
        'Best: Refactor',
        'Reordering Imports',
        'Import-Configure-Run',
        'Dynamic Import'
    ],
    'Code Changes': ['High', 'Low', 'Medium', 'Low'],
    'Maintainability': ['Excellent', 'Poor', 'Good', 'Good'],
    'PEP 8 Compliant': ['Yes', 'No', 'Yes', 'Yes'],
    'When to Use': [
        'New code or major refactor',
        'Never (avoid)',
        'Well-structured applications',
        'Quick fixes, legacy code'
    ]
}

df = pd.DataFrame(comparison_data)
print("\nCircular Dependency Solutions Comparison:")
print("=" * 80)
print(df.to_string(index=False))

### Key Takeaways - Item 88

**✓ Remember:**
- Circular dependencies cause import-time crashes
- Best solution: Refactor to shared utility module
- Practical solution: Dynamic imports
- Avoid: Import reordering

---

## Item 89: Consider warnings to Refactor and Migrate Usage

### The Challenge: API Evolution

As codebases grow, API changes become difficult to coordinate across many dependent systems. The `warnings` module provides a programmatic way to notify users about deprecated usage.

### Example: Evolving an API

Let's start with a simple distance calculation function:

In [ ]:
# Original implementation - implicit units
def print_distance_v1(speed, duration):
    """Calculate distance (assumes mph and hours)"""
    distance = speed * duration
    print(f'{distance} miles')

# Works fine for intended use
print("Car traveling 5 mph for 2.5 hours:")
print_distance_v1(5, 2.5)

# But fails silently for other units
print("\nBullet at 1000 m/s for 3 seconds:")
print_distance_v1(1000, 3)  # Wrong! Gives 3000 miles instead of 3000 meters
print("(Should be 3000 meters, not 3000 miles!)")

### Improved API with Explicit Units

In [ ]:
# Unit conversion constants
CONVERSIONS = {
    'mph': 1.60934 / 3600 * 1000,  # to m/s
    'hours': 3600,                  # to seconds
    'miles': 1.60934 * 1000,       # to meters
    'meters': 1,                    # base unit
    'm/s': 1,                       # base unit
    'seconds': 1,                   # base unit
}

def convert(value, units):
    """Convert to base units (meters, seconds)"""
    rate = CONVERSIONS[units]
    return rate * value

def localize(value, units):
    """Convert from base units to desired units"""
    rate = CONVERSIONS[units]
    return value / rate

def print_distance_v2(speed, duration, *,
                      speed_units='mph',
                      time_units='hours',
                      distance_units='miles'):
    """Calculate distance with explicit unit handling"""
    norm_speed = convert(speed, speed_units)
    norm_duration = convert(duration, time_units)
    norm_distance = norm_speed * norm_duration
    distance = localize(norm_distance, distance_units)
    print(f'{distance} {distance_units}')

# Now works correctly
print("Bullet at 1000 m/s for 3 seconds:")
print_distance_v2(1000, 3, 
                  speed_units='m/s',
                  time_units='seconds',
                  distance_units='miles')

### Using Warnings for Migration

How do we migrate existing callers to the new API without breaking their code?

In [ ]:
import warnings

def print_distance_with_warnings(speed, duration, *,
                                 speed_units=None,
                                 time_units=None,
                                 distance_units=None):
    """Distance calculation with deprecation warnings"""
    if speed_units is None:
        warnings.warn(
            'speed_units required',
            DeprecationWarning)
        speed_units = 'mph'
    
    if time_units is None:
        warnings.warn(
            'time_units required',
            DeprecationWarning)
        time_units = 'hours'
    
    if distance_units is None:
        warnings.warn(
            'distance_units required',
            DeprecationWarning)
        distance_units = 'miles'
    
    norm_speed = convert(speed, speed_units)
    norm_duration = convert(duration, time_units)
    norm_distance = norm_speed * norm_duration
    distance = localize(norm_distance, distance_units)
    print(f'{distance} {distance_units}')

# Old code still works but warns
print("Calling with old API (no units specified):")
print_distance_with_warnings(5, 2.5)
print("\n⚠️  User sees deprecation warning in stderr")

### Better Warning Helper with Stack Level

In [ ]:
def require(name, value, default):
    """Helper to warn about missing parameters"""
    if value is not None:
        return value
    
    warnings.warn(
        f'{name} will be required soon, update your code',
        DeprecationWarning,
        stacklevel=3)  # Points to actual caller, not this function
    return default

def print_distance_improved(speed, duration, *,
                           speed_units=None,
                           time_units=None,
                           distance_units=None):
    """Cleaner warning implementation"""
    speed_units = require('speed_units', speed_units, 'mph')
    time_units = require('time_units', time_units, 'hours')
    distance_units = require('distance_units', distance_units, 'miles')
    
    norm_speed = convert(speed, speed_units)
    norm_duration = convert(duration, time_units)
    norm_distance = norm_speed * norm_duration
    distance = localize(norm_distance, distance_units)
    print(f'{distance} {distance_units}')

# Much cleaner implementation!
print("Using improved warning helper:")
print_distance_improved(10, 2)

print("\n✓ Advantages:")
print("  - Less boilerplate")
print("  - Proper stack level (shows caller's location)")
print("  - Reusable across functions")

### Configuring Warning Behavior

In [ ]:
# 1. Convert warnings to errors (useful in tests)
print("1. Converting warnings to errors:")
warnings.simplefilter('error')
try:
    warnings.warn('This will raise!', DeprecationWarning)
except DeprecationWarning as e:
    print(f"   Caught exception: {e}")

# 2. Ignore warnings
print("\n2. Ignoring warnings:")
warnings.simplefilter('ignore')
warnings.warn('This will not print')
print("   (Warning was suppressed)")

# 3. Reset to default behavior
print("\n3. Reset to default:")
warnings.resetwarnings()
warnings.simplefilter('default')
print("   Warnings will now print to stderr")

print("\n📝 Command-line control:")
print("   python -W error script.py    # Warnings as errors")
print("   python -W ignore script.py   # Suppress warnings")
print("   PYTHONWARNINGS=error python script.py")

### Integrating Warnings with Logging

In [ ]:
import logging
import io

# Setup logging to capture warnings
fake_stderr = io.StringIO()
handler = logging.StreamHandler(fake_stderr)
formatter = logging.Formatter(
    '%(asctime)s WARNING] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
handler.setFormatter(formatter)

logging.captureWarnings(True)
logger = logging.getLogger('py.warnings')
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

warnings.resetwarnings()
warnings.simplefilter('default')
warnings.warn('This goes to logging system')

print("Warning captured in logs:")
print(fake_stderr.getvalue())

print("\n✓ Benefits of logging integration:")
print("  - Warnings go to existing error reporting")
print("  - Production monitoring catches issues")
print("  - Can track deprecation usage in real deployments")

### Testing Warning Generation

In [ ]:
# Best practice: Test that warnings are raised correctly

with warnings.catch_warnings(record=True) as found_warnings:
    warnings.simplefilter('always')  # Ensure warnings are triggered
    found = require('my_arg', None, 'fake units')
    expected = 'fake units'
    
    # Verify function behavior
    assert found == expected, f"Expected {expected}, got {found}"
    
    # Verify warning was raised
    assert len(found_warnings) == 1, "Should raise exactly one warning"
    
    single_warning = found_warnings[0]
    assert 'my_arg will be required soon' in str(single_warning.message)
    assert single_warning.category == DeprecationWarning
    
    print("✓ All warning tests passed!")
    print(f"  Message: {single_warning.message}")
    print(f"  Category: {single_warning.category.__name__}")

### Warning Types

In [ ]:
# Python provides several warning categories

warning_types = {
    'DeprecationWarning': 'Feature will be removed in future',
    'FutureWarning': 'Behavior will change in future',
    'PendingDeprecationWarning': 'Feature deprecated but removal not scheduled',
    'RuntimeWarning': 'Dubious runtime behavior',
    'SyntaxWarning': 'Dubious syntax',
    'UserWarning': 'Generic user warning',
}

print("Standard Warning Categories:")
print("=" * 60)
for warning_type, description in warning_types.items():
    print(f"{warning_type:30s} {description}")

# Demonstrate different types
print("\n" + "=" * 60)
warnings.simplefilter('always')

print("\nDeprecationWarning example:")
warnings.warn('This API is deprecated', DeprecationWarning, stacklevel=2)

print("\nFutureWarning example:")
warnings.warn('This behavior will change in Python 4.0', FutureWarning, stacklevel=2)

### Key Takeaways - Item 89

**✓ Remember:**
- Use `warnings` for API migration guidance
- `stacklevel` parameter shows the right location
- `-W error` flag turns warnings into errors for testing
- Integrate warnings with logging for production monitoring
- Write tests to verify warning generation

---

## Item 90: Consider Static Analysis via typing to Obviate Bugs

### Introduction to Python Type Hints

Python's `typing` module enables gradual typing - you can add type annotations incrementally to catch bugs through static analysis.

### Basic Type Annotations

In [ ]:
# Without type hints - bugs not caught until runtime
def subtract_v1(a, b):
    return a - b

# This compiles but fails at runtime
try:
    result = subtract_v1(10, '5')
except TypeError as e:
    print(f"Runtime error: {e}")

# With type hints - mypy catches this before runtime
def subtract_v2(a: int, b: int) -> int:
    return a - b

# Correct usage
result = subtract_v2(10, 5)
print(f"\nCorrect usage: 10 - 5 = {result}")

# mypy would catch this error:
# result = subtract_v2(10, '5')  # error: Argument 2 has incompatible type "str"

print("\n✓ Type hints enable:")
print("  - Static error detection")
print("  - Better IDE autocomplete")
print("  - Self-documenting code")

### Common Type Annotation Patterns

In [ ]:
from typing import List, Dict, Tuple, Optional, Union, Any

# Basic types
def process_data(count: int, name: str, active: bool) -> None:
    """Basic parameter and return type annotations"""
    print(f"{name}: {count} items, active={active}")

# Collections
def sum_numbers(numbers: List[int]) -> int:
    """List of specific type"""
    return sum(numbers)

def get_config() -> Dict[str, Any]:
    """Dictionary with string keys, any value type"""
    return {'timeout': 30, 'retry': True}

def parse_point(data: str) -> Tuple[float, float]:
    """Returns tuple of specific size and types"""
    x, y = data.split(',')
    return float(x), float(y)

# Optional types (can be None)
def find_user(user_id: int) -> Optional[str]:
    """May return None if user not found"""
    users = {1: 'Alice', 2: 'Bob'}
    return users.get(user_id)

# Union types (multiple possible types)
def process_id(user_id: Union[int, str]) -> str:
    """Accepts int or str"""
    return str(user_id)

# Demonstrate usage
process_data(42, "items", True)
print(f"Sum: {sum_numbers([1, 2, 3, 4, 5])}")
print(f"Config: {get_config()}")
print(f"Point: {parse_point('3.14,2.71')}")
print(f"User: {find_user(1)}")
print(f"ID: {process_id(123)}")

### Class Type Annotations

In [ ]:
class Counter:
    """Counter with type annotations"""
    
    def __init__(self) -> None:
        self.value: int = 0  # Field annotation
    
    def add(self, offset: int) -> None:
        self.value += offset
    
    def get(self) -> int:
        return self.value

# Usage
counter = Counter()
counter.add(5)
counter.add(3)
result = counter.get()
print(f"Counter value: {result}")
assert result == 8

# Common mistakes that mypy would catch:
# counter.add("5")           # error: Argument has incompatible type "str"
# x: str = counter.get()     # error: Incompatible types in assignment

print("\n✓ Type hints catch:")
print("  - Wrong argument types")
print("  - Wrong return type usage")
print("  - Missing return statements")

### Generic Types and TypeVar

In [ ]:
from typing import TypeVar, Callable

# Generic type that works with any type
T = TypeVar('T')

def get_first(items: List[T]) -> T:
    """Generic function - preserves type information"""
    return items[0]

# Type checker knows the return type matches input
numbers: List[int] = [1, 2, 3]
first_number: int = get_first(numbers)  # Type checker: ✓

names: List[str] = ['Alice', 'Bob']
first_name: str = get_first(names)      # Type checker: ✓

print(f"First number: {first_number}")
print(f"First name: {first_name}")

# Constrained TypeVar - limited to specific types
Number = TypeVar('Number', int, float)

def add(x: Number, y: Number) -> Number:
    """Works with int or float, not str"""
    return x + y  # type: ignore

print(f"\nAdd integers: {add(5, 3)}")
print(f"Add floats: {add(3.14, 2.71)}")
# add("hello", "world")  # mypy error: Type must be int or float

### Advanced: Callable Types

In [ ]:
from typing import Callable, List, TypeVar

Value = TypeVar('Value')

# Callable[argument_types, return_type]
def combine(func: Callable[[Value, Value], Value], 
           values: List[Value]) -> Value:
    """Generic reducer function with type safety"""
    assert len(values) > 0
    result = values[0]
    for next_value in values[1:]:
        result = func(result, next_value)
    return result

def add_ints(x: int, y: int) -> int:
    return x + y

def multiply_ints(x: int, y: int) -> int:
    return x * y

numbers = [1, 2, 3, 4, 5]
sum_result = combine(add_ints, numbers)
product_result = combine(multiply_ints, numbers)

print(f"Sum: {sum_result}")
print(f"Product: {product_result}")

# Type checker would catch:
# combine(add_ints, [1, 2, 3, 4j])  # error: complex not compatible with int

print("\n✓ Callable types ensure:")
print("  - Function signatures match expectations")
print("  - Argument types are consistent")
print("  - Return types are compatible")

### Optional Types and None Checking

In [ ]:
from typing import Optional

# Without type hints - None bugs are common
def get_or_default_buggy(value, default):
    if value is not None:
        return value
    return value  # BUG: should return default!

result = get_or_default_buggy(None, 5)
print(f"Buggy version returns: {result} (should be 5)")

# With type hints - mypy catches the bug
def get_or_default(value: Optional[int], default: int) -> int:
    if value is not None:
        return value
    return default  # Fixed!

result = get_or_default(3, 5)
print(f"\nWith value: {result}")

result = get_or_default(None, 5)
print(f"Without value: {result}")

# Type checker enforces null safety
print("\n✓ Optional types prevent:")
print("  - Null reference errors")
print("  - Forgotten None checks")
print("  - Incorrect default handling")

### Forward References

In [ ]:
from __future__ import annotations

# Without __future__ import, this would fail
class FirstClass:
    def __init__(self, value: SecondClass) -> None:
        """Can reference SecondClass before it's defined"""
        self.value = value

class SecondClass:
    def __init__(self, value: int) -> None:
        self.value = value

# Usage works fine
second = SecondClass(5)
first = FirstClass(second)
print(f"Value: {first.value.value}")

print("\n✓ from __future__ import annotations:")
print("  - Enables forward references")
print("  - Improves startup performance")
print("  - Will be default in Python 4.0")
print("\nAlternative: Use string literal")
print("  def __init__(self, value: 'SecondClass') -> None")

### Best Practices for Using Type Hints

In [ ]:
# Best practices demonstration

best_practices = {
    'Strategy': [
        'Write code first',
        'Add tests',
        'Add type hints strategically',
        'Run type checker incrementally'
    ],
    'Where to Apply': [
        'Public APIs',
        'Complex/error-prone code',
        'Integration boundaries',
        'Library interfaces'
    ],
    'Where to Skip': [
        'Small scripts',
        'Prototypes',
        'Internal utilities',
        'Legacy code (unless refactoring)'
    ],
    'Integration': [
        'Add mypy to CI/CD pipeline',
        'Store config in repository',
        'Use consistent rules across team',
        'Run incrementally during development'
    ]
}

print("Type Hints Best Practices")
print("=" * 70)
for category, practices in best_practices.items():
    print(f"\n{category}:")
    for i, practice in enumerate(practices, 1):
        print(f"  {i}. {practice}")

print("\n" + "=" * 70)
print("\n⚠️  Important Notes:")
print("  • Don't aim for 100% coverage (diminishing returns)")
print("  • Type hints are for developers, not runtime")
print("  • Multiple tools available: mypy, pyright, pytype, pyre")
print("  • Exceptions are NOT type-checked in Python")

### Running mypy (Command-line Examples)

In [ ]:
# mypy command-line usage examples

mypy_examples = """
Basic Usage:
============
# Check single file
$ python3 -m mypy script.py

# Check with strict mode (recommended)
$ python3 -m mypy --strict script.py

# Check entire project
$ python3 -m mypy src/

Common Flags:
=============
--strict              # Enable all optional checks
--ignore-missing-imports  # Don't error on untyped imports
--disallow-untyped-defs   # Require type hints on functions
--warn-return-any         # Warn when returning Any type
--no-implicit-optional    # Disallow implicit Optional

Configuration File (mypy.ini):
==============================
[mypy]
python_version = 3.9
warn_return_any = True
warn_unused_configs = True
disallow_untyped_defs = True

[mypy-third_party.*]
ignore_missing_imports = True
"""

print(mypy_examples)

print("\n📋 Workflow:")
print("  1. Write code without types")
print("  2. Add tests")
print("  3. Add type hints to public API")
print("  4. Run mypy incrementally")
print("  5. Fix errors as you go")
print("  6. Add to CI/CD pipeline")

### Comparison: Type Checking Tools

In [ ]:
import pandas as pd

tools_comparison = {
    'Tool': ['mypy', 'pyright', 'pytype', 'pyre'],
    'Developer': ['Python Core', 'Microsoft', 'Google', 'Meta'],
    'Speed': ['Medium', 'Fast', 'Slow', 'Fast'],
    'Strictness': ['High', 'Very High', 'Medium', 'High'],
    'Inference': ['Limited', 'Good', 'Excellent', 'Good'],
    'Best For': [
        'General use, most popular',
        'VS Code, performance',
        'Type inference, gradual typing',
        'Large codebases, speed'
    ]
}

df = pd.DataFrame(tools_comparison)
print("\nPython Type Checking Tools Comparison:")
print("=" * 100)
print(df.to_string(index=False))
print("=" * 100)
print("\n💡 Recommendation: Start with mypy --strict")

### Real-World Example: Type-Safe Data Processing

In [ ]:
from typing import List, Dict, Optional, Callable
from dataclasses import dataclass

@dataclass
class User:
    """Type-safe user data structure"""
    id: int
    name: str
    email: str
    age: Optional[int] = None

class UserRepository:
    """Type-safe user repository"""
    
    def __init__(self) -> None:
        self._users: Dict[int, User] = {}
    
    def add(self, user: User) -> None:
        """Add user to repository"""
        self._users[user.id] = user
    
    def get(self, user_id: int) -> Optional[User]:
        """Get user by ID"""
        return self._users.get(user_id)
    
    def filter(self, predicate: Callable[[User], bool]) -> List[User]:
        """Filter users by predicate"""
        return [user for user in self._users.values() if predicate(user)]

# Usage with complete type safety
repo = UserRepository()

# Add users
repo.add(User(1, "Alice", "alice@example.com", 30))
repo.add(User(2, "Bob", "bob@example.com", 25))
repo.add(User(3, "Charlie", "charlie@example.com"))

# Type-safe retrieval
user: Optional[User] = repo.get(1)
if user:
    print(f"Found: {user.name} ({user.email})")

# Type-safe filtering
adults: List[User] = repo.filter(lambda u: u.age is not None and u.age >= 18)
print(f"\nAdults: {[u.name for u in adults]}")

print("\n✓ Type safety provides:")
print("  - Compile-time error detection")
print("  - Better IDE support")
print("  - Self-documenting code")
print("  - Refactoring confidence")

### Key Takeaways - Item 90

**✓ Remember:**
- Type hints enable static analysis without runtime cost
- Multiple tools available: mypy, pyright, pytype, pyre
- Most valuable at API boundaries
- Apply incrementally - don't aim for 100% coverage
- Include type checking in CI/CD pipeline
- Exceptions are NOT type-checked

---

## Chapter 10 Summary (Items 88-90)

### Integration: Using All Three Together

In [ ]:
# Comprehensive example using all three concepts

from typing import Optional
import warnings

# Module A: Core functionality
class DataProcessor:
    """Type-safe data processor"""
    
    def process(self, data: str) -> int:
        """Process data and return count"""
        return len(data)

# Module B: Uses Module A with proper patterns
class DataAnalyzer:
    """Analyzer avoiding circular dependencies"""
    
    def __init__(self) -> None:
        self._processor: Optional[DataProcessor] = None
    
    def analyze(self, data: str, format: Optional[str] = None) -> dict:
        """Analyze data with deprecation warning"""
        # Dynamic import to avoid circular dependency
        if self._processor is None:
            self._processor = DataProcessor()
        
        # Deprecation warning for old API
        if format is None:
            warnings.warn(
                'format parameter will be required in future versions',
                DeprecationWarning,
                stacklevel=2
            )
            format = 'json'
        
        # Type-safe processing
        count = self._processor.process(data)
        return {'count': count, 'format': format}

# Usage
analyzer = DataAnalyzer()

# Modern API (no warning)
result = analyzer.analyze("test data", format='json')
print(f"Modern API: {result}")

# Legacy API (with warning)
warnings.simplefilter('always')
result = analyzer.analyze("legacy data")
print(f"Legacy API: {result}")

print("\n✓ Combined benefits:")
print("  • No circular dependencies (dynamic import)")
print("  • Clear migration path (warnings)")
print("  • Type safety (static analysis)")
print("  • Backward compatibility (optional params)")

### Final Recommendations

In [ ]:
recommendations = """
Collaboration Best Practices Checklist
=======================================

Architecture:
□ Design module dependencies carefully
□ Use dynamic imports as last resort for circular deps
□ Prefer shared utility modules at dependency tree bottom

API Evolution:
□ Add deprecation warnings before breaking changes
□ Use warnings.warn with proper stacklevel
□ Test warning generation
□ Document migration path clearly
□ Integrate warnings with logging in production

Type Safety:
□ Add type hints to public APIs first
□ Run type checker in CI/CD pipeline
□ Store mypy configuration in repository
□ Apply types incrementally, not all at once
□ Focus on error-prone code areas

Testing:
□ Test circular import scenarios
□ Verify warning messages
□ Use -W error in test environment
□ Write type-checked integration tests
"""

print(recommendations)

print("\n📚 Related Topics:")
print("  • Item 87: Define Root Exceptions")
print("  • Item 84: Write Docstrings")
print("  • Item 76: TestCase Subclasses")
print("  • Item 2: Follow PEP 8")

---

## Practice Exercises

Try these exercises to reinforce the concepts:

In [ ]:
# Exercise 1: Fix circular dependency
print("Exercise 1: Refactor this circular dependency using dynamic imports")
print("""
# module_a.py
import module_b
class A:
    def use_b(self):
        return module_b.B()

# module_b.py  
import module_a
class B:
    def use_a(self):
        return module_a.A()
""")

# Exercise 2: Add deprecation warnings
print("\nExercise 2: Add deprecation warnings to this function")
print("""
def calculate(x, y, operation='add'):
    # Deprecate operation parameter
    # Make it required in future
    pass
""")

# Exercise 3: Add type hints
print("\nExercise 3: Add complete type hints")
print("""
def process_items(items, filter_func, transform_func):
    filtered = [item for item in items if filter_func(item)]
    return [transform_func(item) for item in filtered]
""")

---

## Conclusion

These three items form a comprehensive toolkit for professional Python collaboration:

1. **Item 88**: Manage module dependencies properly
2. **Item 89**: Communicate changes through warnings  
3. **Item 90**: Prevent bugs through static analysis

Together, they enable scalable, maintainable, and robust Python codebases that multiple developers can work on effectively.